In [ ]:
from aavomics import database
import anndata
import os
from plotly import graph_objects
from plotly import offline as plotly

In [ ]:
ANNDATA_FILE_NAME = "aavomics_mouse_cortex_2021.h5ad"

In [ ]:
adata = anndata.read_h5ad(os.path.join(database.DATA_PATH, ANNDATA_FILE_NAME))

In [ ]:
CELL_SET_NAMES = ["20181127_TC1", "20190319_TC2", "20190111_BC1", "20190321_BC2", "20190711_TC4", "20190712_TC5", "20190713_TC6", "20190713_TC7", "20200720_BC4_1", "20200720_BC4_2"]
sample_to_virus_to_vegetable = {}
for cell_set_name in CELL_SET_NAMES:
    cell_set = database.CELL_SETS_DICT[cell_set_name]
    if cell_set_name not in sample_to_virus_to_vegetable.keys():
        sample_to_virus_to_vegetable[cell_set_name] = {}
    for injection in cell_set.source_tissue.animal.injections:
        for vector in injection.vector_pool.vectors:
            #print(cell_set_name, vector.name.split())
            vname = vector.name.split()
            sample_to_virus_to_vegetable[cell_set_name][vname[0]]=vname[1]

In [ ]:
neon, cherry, tomato, ruby = [[],[]], [[],[]], [[],[]], [[],[]]
neon_norm, cherry_norm, tomato_norm, ruby_norm = [[],[]], [[],[]], [[],[]], [[],[]]
viruses = ['PHP.eB', 'PHP.V1', 'CAP-B10']
for cell_set_name in CELL_SET_NAMES:
    virus_adata = adata[adata.obs["Cell Set"] == cell_set_name].copy()
    
    for v in viruses:
        if v in virus_adata.obs.columns and virus_adata.obs[v].sum() > 0:
            virus_counts = virus_adata.obs[v].sum()
            norm_factor = virus_adata.X.sum()
            veggie = sample_to_virus_to_vegetable[cell_set_name][v]
            if veggie == 'mNeonGreen':
                neon[0].append(v)
                neon[1].append(virus_counts)
                neon_norm[0].append(v)
                neon_norm[1].append(virus_counts / norm_factor)
            elif veggie == 'mCherry':
                cherry[0].append(v)
                cherry[1].append(virus_counts)
                cherry_norm[0].append(v)
                cherry_norm[1].append(virus_counts / norm_factor)
            elif veggie == 'tdTomato':
                tomato[0].append(v)
                tomato[1].append(virus_counts)
                tomato_norm[0].append(v)
                tomato_norm[1].append(virus_counts / norm_factor)
            elif veggie == 'mRuby2':
                ruby[0].append(v)
                ruby[1].append(virus_counts)
                ruby_norm[0].append(v)
                ruby_norm[1].append(virus_counts / norm_factor)
    print(neon, cherry, tomato, ruby)
    print(neon_norm, cherry_norm, tomato_norm, ruby_norm)

In [ ]:
# Normalized Counts
trace_1 = graph_objects.Scatter(
    x=neon_norm[0],
    y=neon_norm[1],
    mode="markers",
    name='mNeonGreen',
    marker={
        "size": 10
    }
)
trace_2 = graph_objects.Scatter(
    x=cherry_norm[0],
    y=cherry_norm[1],
    mode="markers",
    name='mCherry',
    marker={
        "size": 10
    }
)
trace_3 = graph_objects.Scatter(
    x=tomato_norm[0],
    y=tomato_norm[1],
    mode="markers",
    name='tdTomato',
    marker={
        "size": 10
    }
)
trace_4 = graph_objects.Scatter(
    x=ruby_norm[0],
    y=ruby_norm[1],
    mode="markers",
    name='mRuby2',
    marker={
        "size": 10
    }
)

layout = {}

layout["height"] = 600
layout["width"] = 600
layout["plot_bgcolor"] = "rgba(255, 255, 255, 0)"
layout["paper_bgcolor"] = "rgba(255, 255, 255, 0)"
layout["yaxis"] = {
    "title": {
        "text": "Transcript Abundance",
    },
    "gridcolor": "rgba(0, 0, 0, 0.25)",
    "type": "log"
}
layout["xaxis"] = {
    "title": "Virus"
}
layout["title"] = {
    "text":"Transcript Abundance by Fluorophore"
}

figure = graph_objects.Figure(data=[trace_3,trace_2,trace_1,trace_4], layout=layout)

plotly.iplot(figure)

figure.write_image(os.path.join("out", "transcript_abundance_by_fluorophore.svg"))